In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import streamlit as st
import altair as alt
from utils import fetch_odds, calculate_ev, plus_prefix, fix_names

In [2]:
#  def main : displays front ui

odds = fetch_odds('win').dropna()

In [5]:
books = ['betmgm', 'betfair', 'fanduel', 'draftkings', 'bovada',
       'williamhill', 'betonline', 'betcris', 'unibet', 'caesars', 'bet365',
       'betway', 'pinnacle', 'skybet', 'pointsbet']

In [27]:
aggregate_lines = odds[books].T.mean().round(0).to_frame()

In [38]:
def calc_ev(line):
    return (line/100)+1


In [57]:
df = odds[['player_name','datagolf_base_history_fit']].merge(
        aggregate_lines,
        left_index=True, 
        right_index=True
        ).rename(
            columns={
                'player_name':'player',
                'datagolf_base_history_fit':'real_odds',
                0:'agg_line'
                }
                )

df['dec_agg_line'] = (df['agg_line'] / 100) / 1
df['dec_real_odds'] = (df['real_odds'] / 100) / 1

df['ev'] = round(df['dec_real_odds'] * (1/df['dec_agg_line']) - 1,2)

df.drop(columns = ['dec_agg_line','dec_real_odds'], inplace=True)

df['real_odds'].apply(plus_prefix)
df['agg_line'].apply(plus_prefix)
df

,player,real_odds,agg_line,ev
0,"Scheffler, Scottie",584.0,430.0,0.36
1,"McIlroy, Rory",1348.0,1065.0,0.27
2,"Schauffele, Xander",1362.0,1401.0,-0.03
3,"Rahm, Jon",1600.0,1167.0,0.37
4,"Matsuyama, Hideki",2530.0,1810.0,0.40
...,...,...,...,...
71,"Dunlap, Nick",111011.0,29975.0,2.70
73,"Watson, Bubba",115285.0,30119.0,2.83
74,"Woods, Tiger",117547.0,12948.0,8.08
76,"Schwartzel, Charl",139435.0,38875.0,2.59


In [45]:
round(df['dec_real_odds'] * (1/df['dec_agg_line']) - 1,2)

0      0.36
1      0.27
2     -0.03
3      0.37
4      0.40
      ...  
71     2.70
73     2.83
74     8.08
76     2.59
77    12.08
Length: 74, dtype: float64

In [ ]:
pd.concat([fetch_odds('win'), fetch_odds('top_5'), fetch_odds('top_10'), fetch_odds('top_20')]).dropna()

In [ ]:
odds = fetch_odds('win')
odds = pd.concat([odds, fetch_odds('top_5'), fetch_odds('top_10'), fetch_odds('top_20')]).dropna()

odds.rename(columns={'datagolf_base_history_fit':'real_odds'},inplace=True)
odds['player_name'] = fix_names(odds)

odds = calculate_ev(odds)

odds = odds[['market', 'player_name', 'real_odds', 'fanduel','fanduel_ev', 'draftkings','draftkings_ev', 'betmgm','betmgm_ev']].convert_dtypes()

odds['real_odds'] = odds['real_odds'].dropna().apply(plus_prefix)
odds['fanduel'] = odds['fanduel'].dropna().apply(plus_prefix)
odds['draftkings'] = odds['draftkings'].dropna().apply(plus_prefix)
odds['betmgm'] = odds['betmgm'].dropna().apply(plus_prefix)

odds['market'] = odds['market'].replace({'win': 'Win', 'top_5': 'Top 5', 'top_10': 'Top 10', 'top_20': 'Top 20'})

odds.columns = ['Market','Player','True Odds','FD','FD EV','DK','DK EV','MGM','MGM EV']